In [8]:
# https://www.quantinsti.com/blog/sentiment-analysis-news-python/

In [1]:
import csv
import time
import requests
from bs4 import BeautifulSoup
from pattern.en import ngrams

Base_url = "http://www.moneycontrol.com"

# Build a dictionary of companies and their abbreviated names 
companies = {'cadilahealthcare':'CHC','piramalenterprises':'PH05',
             'glenmarkpharma':'GP08','glaxosmithklinepharmaceuticals':'GSK',
             'sunpharmaceuticalindustries':'SPI','lupinlaboratories':'LL',
             'cipla':'C','aurobindopharma':'AP',
             'drreddyslaboratories':'DRL','divislaboratories':'DL03'}
             
# Create a list of the news section urls of the respective companies 
url_list = ['http://www.moneycontrol.com/company-article/{}/news/{}#{}'.format(k,v,v) for k,v in companies.iteritems()]
print url_list

['http://www.moneycontrol.com/company-article/cipla/news/C#C', 'http://www.moneycontrol.com/company-article/drreddyslaboratories/news/DRL#DRL', 'http://www.moneycontrol.com/company-article/aurobindopharma/news/AP#AP', 'http://www.moneycontrol.com/company-article/sunpharmaceuticalindustries/news/SPI#SPI', 'http://www.moneycontrol.com/company-article/piramalenterprises/news/PH05#PH05', 'http://www.moneycontrol.com/company-article/divislaboratories/news/DL03#DL03', 'http://www.moneycontrol.com/company-article/lupinlaboratories/news/LL#LL', 'http://www.moneycontrol.com/company-article/cadilahealthcare/news/CHC#CHC', 'http://www.moneycontrol.com/company-article/glenmarkpharma/news/GP08#GP08', 'http://www.moneycontrol.com/company-article/glaxosmithklinepharmaceuticals/news/GSK#GSK']


In [2]:
# Create an empty list which will contain the selected news articles 
List_of_links = []   

# Extract the relevant news articles weblinks from the news section of selected companies
for urls in url_list:
   html = requests.get(urls)
   soup = BeautifulSoup(html.text,'html.parser') # Create a BeautifulSoup object 

   # Retrieve a list of all the links and the titles for the respective links
   word1,word2,word3 = "US","USA","USFDA"
 
   sub_links = soup.find_all('a', class_='arial11_summ')
   for links in sub_links:
      sp = BeautifulSoup(str(links),'html.parser')  # first convert into a string
      tag = sp.a
      #print tag
      if word1 in tag['title'] or word2 in tag['title'] or word3 in tag['title']:
          category_links = Base_url + tag["href"]
          List_of_links.append(category_links)
          time.sleep(3)
          
# Print the select list of news articles weblinks    
#for p in List_of_links: print p

In [3]:
List_of_links

[u'http://www.moneycontrol.com/news/fiiview/rising-us-approvals-to-boost-earnings-for-pharma-idfc-sec_7970661.html',
 u'http://www.moneycontrol.com/news/business/dr-reddys-launches-raloxifene-hydrochloride-tabletsus_7951381.html',
 u'http://www.moneycontrol.com/news/announcements/dr-reddys-lab-launches-raloxifene-tabletsus-market_7950241.html',
 u'http://www.moneycontrol.com/news/fiiview/rising-us-approvals-to-boost-earnings-for-pharma-idfc-sec_7970661.html',
 u'http://www.moneycontrol.com/news/business/sun-pharma-cadila-healthcare-recall-productsus-market_8043201.html',
 u'http://www.moneycontrol.com/news/business/sun-pharma-unveils-first-branded-ophthalmic-productus_8029781.html',
 u'http://www.moneycontrol.com/news/business/sun-pharma-cadila-healthcare-recall-productsus-market_8043201.html',
 u'http://www.moneycontrol.com/news/business/cadilas-20k-bottlesanti-depressant-recalledus_8010281.html',
 u'http://www.moneycontrol.com/news/buzzingstocks/sun-pharma-falls-7-sector-under-pressu

In [4]:
# Remove the duplicate news articles based on News Title
unique_links = list(set(List_of_links))
for q in unique_links: print q

# Create a dictionary of positive/negative words related to the Pharma Sector
reader = csv.reader(open('dict.csv', 'r'))

http://www.moneycontrol.com/news/buzzingstocks/sun-pharma-falls-7-sector-under-pressureus-probe-fear_7896501.html
http://www.moneycontrol.com/news/business/sun-pharma-unveils-first-branded-ophthalmic-productus_8029781.html
http://www.moneycontrol.com/news/business/dr-reddys-launches-raloxifene-hydrochloride-tabletsus_7951381.html
http://www.moneycontrol.com/news/business/sun-pharma-cadila-healthcare-recall-productsus-market_8043201.html
http://www.moneycontrol.com/news/fiiview/rising-us-approvals-to-boost-earnings-for-pharma-idfc-sec_7970661.html
http://www.moneycontrol.com/news/business/cadilas-20k-bottlesanti-depressant-recalledus_8010281.html
http://www.moneycontrol.com/news/announcements/dr-reddys-lab-launches-raloxifene-tabletsus-market_7950241.html


In [5]:
!head dict1.csv

head: cannot open `dict1.csv' for reading: No such file or directory


In [6]:
pharma_dict = dict((rows[0],rows[1]) for rows in reader)

# Creating an empty list which will be filled later with news article links, and Polarity values (pos/neg)
df =[]
print df

[]


In [7]:
# Open the choosen news articles and extract the main text  
for selected_links in unique_links:
   results_url = selected_links 
   #print results_url
   
   results = requests.get(results_url)
   results_text = BeautifulSoup(results.text)
   extract_text = results_text.find(class_='arti_cont')
   final_text = extract_text.get_text()
   final_text1 = ngrams(final_text, n=1, punctuation=".,;:!?()[]{}`''\"@#$^&amp;*+-|=~_", continuous=False)
  # Checking if any of the words in the news article text matches with the words in the Pharma dictionary(pos/neg)
   new_dict = {}
   positive_score,negative_score = 0,0
   
   for x in final_text1:
      if pharma_dict.has_key(x[0]):
         new_dict[x[0]] = pharma_dict[x[0]] 
                  
   positive_list = [] ; negative_list = [];
   for key, value in new_dict.items():
      if value == 'positive': positive_list.append(key)
      if value == 'negative': negative_list.append(key)
                           
   # Compute the positive score, the negative score for each news articles
   positive_score = (positive_list) ; negative_score = (negative_list);
   #print positive_list ; print negative_list ;
   #print positive_score ; print negative_score;
   # Appending the empty list to create the final Sentiment analysis output
   var_nos = [results_url,positive_score, negative_score]
   df.append(var_nos)
   
# Print the final list of the Sentiment Analysis 
for item in df:
      print(item)

[u'http://www.moneycontrol.com/news/buzzingstocks/sun-pharma-falls-7-sector-under-pressureus-probe-fear_7896501.html', [], []]
[u'http://www.moneycontrol.com/news/business/sun-pharma-unveils-first-branded-ophthalmic-productus_8029781.html', [], []]
[u'http://www.moneycontrol.com/news/business/dr-reddys-launches-raloxifene-hydrochloride-tabletsus_7951381.html', [], []]
[u'http://www.moneycontrol.com/news/business/sun-pharma-cadila-healthcare-recall-productsus-market_8043201.html', [], []]
[u'http://www.moneycontrol.com/news/fiiview/rising-us-approvals-to-boost-earnings-for-pharma-idfc-sec_7970661.html', [], []]
[u'http://www.moneycontrol.com/news/business/cadilas-20k-bottlesanti-depressant-recalledus_8010281.html', [], [u'failed']]
[u'http://www.moneycontrol.com/news/announcements/dr-reddys-lab-launches-raloxifene-tabletsus-market_7950241.html', [], []]


/opt/conda/lib/python2.7/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 162 of the file /opt/conda/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "html.parser")

  markup_type=markup_type))
